In [5]:
import requests
import json

subject = 'Chùa'
url = 'https://vi.wikipedia.org/w/api.php'
params = {
        'action': 'query',
        'format': 'json',
        'titles': subject,
        'prop': 'extracts',
        'exintro': True,
        'explaintext': True,
    }

response = requests.get(url, params=params)
data = response.json()

page = next(iter(data['query']['pages'].values()))
print(page)

{'pageid': 8813, 'ns': 0, 'title': 'Chùa', 'extract': 'Chùa là một công trình kiến trúc phục vụ mục đích tín ngưỡng. Chùa được xây dựng phổ biến ở các nước Nam Á như Ấn Độ, Nepal, Bhutan ,... cùng với một số nước Đông Á và Đông Nam Á như Trung Quốc, Nhật Bản, Việt Nam và thường là nơi thờ Phật. Tại nhiều nơi, chùa có nhiều điểm giống với chùa tháp của Ấn Độ, vốn là nơi cất giữ Xá-lị và chôn cất các vị đại sư, thường có nhiều tháp bao xung quanh. Chùa là nơi tiêu biểu cho Chân như, được nhân cách hóa bằng hình tượng một đức Phật được thờ ngay giữa chùa. Nhiều chùa được thiết kế như một Man-đa-la, gồm một trục ở giữa với các vị Phật ở bốn phương. Cũng có nhiều chùa có nhiều tầng, đại diện cho Ba thế giới (tam giới), các cấp bậc tiêu biểu cho Thập địa của Bồ Tát. Có nhiều chùa được xây tám mặt đại diện cho Pháp luân hoặc Bát chính đạo.Chùa còn là nơi tập trung của các sư, tăng, (hay ni nếu là chùa nữ) sinh hoạt, tu hành, và thuyết giảng đạo Phật. Tại nơi này, mọi người kể cả tín đồ hay ng

In [10]:
import requests
import json

from IPython.display import Image
from IPython.core.display import HTML

import pandas as pd

In [18]:
def infobox_dict(url):
    response = requests.get(url)
    #Tập_tin:Chùa_Lý_Quốc_Sư.JPG
    data = response.json()

    # text = json.dumps(data, indent=2, ensure_ascii=False)
    # print(data['source'])
    # Image(url= "https://commons.wikimedia.org/wiki/Special:FilePath/Chùa Lý Quốc Sư.JPG", width=400)
    # print("no")

    revisions = data['source']

    start = revisions.find("{{")
    end = revisions.find("\n}}", start)
    infobox = revisions[start:end]

    # Parse the infobox template into a dictionary
    d = {}
    for line in infobox.split("\n|"):
        if "=" in line:
            key, value = line.split("=", 1)
            val = value.strip()
            if bool(val):
                d[key.strip()] = val
            # d[key.strip()] = value.strip() #.replace('[','').replace(']','')

    import pprint
    # Print the resulting dictionary
    # pprint.pprint(d)
    return d

In [19]:
list_dict = []
list_chua = ['Chùa_Láng', 'Chùa_Một_Cột', 'Chùa_Hà']

for loc in list_chua: 
    list_dict.append(infobox_dict('https://vi.wikipedia.org/w/rest.php/v1/page/' + loc))

list_dict

[{'name': 'Chùa Láng',
  'tên khác': 'Pagode des Dames',
  'img': 'Cổng chùa Láng, Hà Nội 002.JPG',
  'img_size': '250px',
  'img_capt': 'Cổng chùa Láng',
  'thờ phụng': '[[Từ Đạo Hạnh]]',
  'founded': 'thời vua [[Lý Anh Tông]]',
  'founder': 'vua [[Lý Anh Tông]]',
  'director': '[[Giáo hội Phật giáo Việt Nam]]',
  'address': '112 phố Chùa Láng, phường Láng Thượng, quận Đống Đa, [[Hà Nội]]',
  'tên tự': 'Chiêu Thiền tự (昭禪寺)',
  'country': '{{VNM}}'},
 {'name': 'Chùa Diên Hựu (延祐寺)\n<br>Chùa Một Cột<br>Chùa Mật<br>',
  'img': 'One Pillar Pagoda Hanoi.jpg',
  'img_capt': 'Liên Hoa Đài',
  'denomination': 'Giáo hội Phật giáo Việt Nam',
  'founded': '1049',
  'founder': '[[Lý Thái Tông]] (1000 - 1054)',
  'abbot': "Đại đức '''Thích Tâm Kiên''' (từ 2007)",
  'address': 'phố Chùa Một Cột, phường [[Đội Cấn, Ba Đình|Đội Cấn]], quận [[Ba Đình]], thành phố [[Hà Nội]]',
  'country': '{{flag|Việt Nam}}'},
 {'name': 'Chùa Hà (Thánh Đức tự)',
  'img': 'ChuaHa TamQuan.JPG',
  'denomination': '[[Giáo

In [20]:
new = pd.DataFrame.from_dict(list_dict)
new

,name,tên khác,img,img_size,img_capt,thờ phụng,founded,founder,director,address,tên tự,country,denomination,abbot
0,Chùa Láng,Pagode des Dames,"Cổng chùa Láng, Hà Nội 002.JPG",250px,Cổng chùa Láng,[[Từ Đạo Hạnh]],thời vua [[Lý Anh Tông]],vua [[Lý Anh Tông]],[[Giáo hội Phật giáo Việt Nam]],"112 phố Chùa Láng, phường Láng Thượng, quận Đố...",Chiêu Thiền tự (昭禪寺),{{VNM}},NaN,NaN
1,Chùa Diên Hựu (延祐寺)\n<br>Chùa Một Cột<br>Chùa ...,NaN,One Pillar Pagoda Hanoi.jpg,NaN,Liên Hoa Đài,NaN,1049,[[Lý Thái Tông]] (1000 - 1054),NaN,"phố Chùa Một Cột, phường [[Đội Cấn, Ba Đình|Độ...",NaN,{{flag|Việt Nam}},Giáo hội Phật giáo Việt Nam,Đại đức '''Thích Tâm Kiên''' (từ 2007)
2,Chùa Hà (Thánh Đức tự),NaN,ChuaHa TamQuan.JPG,NaN,NaN,NaN,Thời vua [[Lý Thánh Tông]],NaN,[[Giáo hội Phật giáo Việt Nam]],"86 phố Chùa Hà, phường Dịch Vọng, quận Cầu Giấ...",NaN,[[Việt Nam]],[[Giáo hội Phật giáo Việt Nam]],NaN


In [39]:
import requests
URL = "https://vi.wikipedia.org/w/api.php"
PARAMS = {
    "action": "query",
    "cmlimit": "100",
    "list": "categorymembers",
    "format": "json"
}

def getAllPages(SET_OF_CATS):
    pageset = set()
    
    for cat in SET_OF_CATS:
          # GET ALL PAGES
        DATA = S.get(url=URL, params={**PARAMS, "cmtitle": cat, "cmtype": "page"}).json()
        PAGES = DATA['query']['categorymembers']
        for page in PAGES:
            pageset.add(page['title'])
    
    return pageset


def recurseCat(S, catset, title, level=0, maxlevel=5):
        # RECURSIVE
    R = S.get(url=URL, params={**PARAMS, "cmtitle": title, "cmtype": "subcat"})
    DATA = R.json()
    # print(DATA)
    PAGES = DATA['query']['categorymembers']

    index = 1
    for page in PAGES:
        print("- "*level,index, page['title'])
        catset.add(page['title'])
        
        if (level < maxlevel):
            recurseCat(S, catset, page['title'], level+1, maxlevel)
        else:
            print("Max level reached")
        index += 1

In [37]:
toptitle = "Thể_loại:Kiến_trúc_cổ_Việt_Nam"
S = requests.Session()
new_set = set()

recurseCat(S, new_set, toptitle,maxlevel=4)

 1 Thể loại:Tháp Chăm
-  1 Thể loại:Đền thờ Ấn Độ giáo tại Việt Nam
-  2 Thể loại:Đền thờ vua chúa Chăm Pa
 2 Thể loại:Chùa Việt Nam
-  1 Thể loại:Chùa Việt Nam theo tỉnh thành
- -  1 Thể loại:Chùa tại An Giang
- -  2 Thể loại:Chùa tại Bạc Liêu
- -  3 Thể loại:Chùa tại Bà Rịa – Vũng Tàu
- -  4 Thể loại:Chùa tại Bắc Giang
- -  5 Thể loại:Chùa tại Bắc Ninh
- -  6 Thể loại:Chùa tại Bến Tre
- -  7 Thể loại:Chùa tại Bình Định
- -  8 Thể loại:Chùa tại Bình Dương
- -  9 Thể loại:Chùa tại Bình Phước
- -  10 Thể loại:Chùa tại Bình Thuận
- -  11 Thể loại:Chùa tại Cần Thơ
- -  12 Thể loại:Chùa tại Cà Mau
- -  13 Thể loại:Chùa tại Đà Nẵng
- -  14 Thể loại:Chùa tại Đắk Lắk
- -  15 Thể loại:Chùa tại Đắk Nông
- -  16 Thể loại:Chùa tại Đồng Tháp
- -  17 Thể loại:Chùa tại Đồng Nai
- -  18 Thể loại:Chùa tại Gia Lai
- -  19 Thể loại:Chùa tại Hà Nam
- -  20 Thể loại:Chùa tại Hà Nội
- -  21 Thể loại:Chùa tại Hà Tĩnh
- -  22 Thể loại:Chùa tại Hải Dương
- -  23 Thể loại:Chùa tại Hải Phòng
- -  24 Thể loại:Ch

In [40]:
new_set

{'Thể loại:Bãi biển Quy Nhơn',
 'Thể loại:Bãi biển Việt Nam',
 'Thể loại:Bãi biển Đà Nẵng',
 'Thể loại:Bảo tàng Lịch sử Quân sự Việt Nam',
 'Thể loại:Bảo tàng Mỹ thuật Việt Nam',
 'Thể loại:Bảo tàng Quân đội nhân dân Việt Nam',
 'Thể loại:Bảo tàng tại Hà Nội',
 'Thể loại:Bảo tàng tại Thành phố Hồ Chí Minh',
 'Thể loại:Bảo tàng tại Việt Nam',
 'Thể loại:Bảo tàng tại Đà Nẵng',
 'Thể loại:Bảo tàng tại Đắk Lắk',
 'Thể loại:Bảo tàng về Chiến tranh Việt Nam',
 'Thể loại:Chùa Việt Nam',
 'Thể loại:Chùa Việt Nam theo tỉnh thành',
 'Thể loại:Chùa Việt Nam tại hải ngoại',
 'Thể loại:Chùa tại An Giang',
 'Thể loại:Chùa tại Bà Rịa – Vũng Tàu',
 'Thể loại:Chùa tại Bình Dương',
 'Thể loại:Chùa tại Bình Phước',
 'Thể loại:Chùa tại Bình Thuận',
 'Thể loại:Chùa tại Bình Định',
 'Thể loại:Chùa tại Bạc Liêu',
 'Thể loại:Chùa tại Bắc Giang',
 'Thể loại:Chùa tại Bắc Ninh',
 'Thể loại:Chùa tại Bến Tre',
 'Thể loại:Chùa tại Cà Mau',
 'Thể loại:Chùa tại Cần Thơ',
 'Thể loại:Chùa tại Gia Lai',
 'Thể loại:Chùa 

In [45]:
len(new_set)


340